In [1]:
cuisine = 'mexican'

def pruning_function(business):
    for category in business['categories']:
        if category.lower() == cuisine:
            return True

In [2]:
import json
import pandas as pd
mexican_restaurants = []
restaurant_ids = set()

with open('../data/business.json') as data_file:    
    all_businesses = [json.loads(line) for line in list(data_file)]
    restaurants_list = list(filter(pruning_function, all_businesses))
    for line in restaurants_list:
        mexican_restaurants.append({'business_id': line['business_id'], 'name': line['name']})
        restaurant_ids.add(line['business_id'])
mexican_restaurants = pd.DataFrame(mexican_restaurants)

In [3]:
import numpy as np
reviews = []
with open('../data/review.json', encoding="utf8") as data_file:
    count = 0
    for line in (data_file):
        row = json.loads(line)
        if row['business_id'] in restaurant_ids:
            reviews.append({'business_id':row['business_id'],'text':row['text']})
reviews = pd.DataFrame(reviews)

In [4]:
topics = ['staff', 'food', 'guacamole', 'queso', 'nacho', 'salsa', 'chicken', 'service', 'carnitas', 'quesadilla', 'tortilla', 'burrito', 'steak']

In [5]:
def clean_review(topic, review, n=4):
    phrases_list = []
    size = len(review)
    for index, word in enumerate(review):
        if word == topic:        
            if index - n >= 0:
                before_portion = review[index-n:index]
            else:
                before_portion = review[0:index]
            if index + n + 1 < size:
                after_portion = review[index:index + n +1]
            else:
                after_portion = review[index:size]
            phrases_list.append(" ".join(before_portion + after_portion).lower())
    return(phrases_list)




In [6]:
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))

def remove_stop_words(review, stop_words = stops):
    return [word for word in review if word not in stops]


In [7]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)

import string
translator =  str.maketrans('', '', string.punctuation)

temp = pd.merge(reviews, mexican_restaurants)
# for review in reviews:
count = 0

for row in temp.groupby('business_id'):
    for review in row[1]['text']:
        for topic in topics:
            if topic in review:
                cleaned_review = clean_review(topic, remove_stop_words(review.translate(translator).split()), 5)

